In [ ]:
!pip install langchain
!pip install -qU langchain-openai
!pip install "langchain<0.1.0"
!pip install unstructured
!pip install faiss-cpu
!pip install transformers
#import the library
import torch
from transformers import DPRQuestionEncoder,DPRQuestionEncoderTokenizer,DPRContextEncoder,DPRContextEncoderTokenizer
from transformers import pipeline,AutoModelForSeq2SeqLM,AutoTokenizer
import faiss
import numpy as np
#load the huggingface model
question_encoder=DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer=DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_encoder=DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer=DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
rag_model=AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
rag_tokenizer=AutoTokenizer.from_pretrained('google/flan-t5-base')
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = DirectoryLoader('/content/knowledge_base/', glob="**/*.txt")
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)
#with open(docs[0], 'r') as f:
#    content = f.read()
#print(content)
content = docs[0].page_content
doc_embeddings=[]
inputs=context_tokenizer(content,return_tensors='pt',padding=True)
embedding=context_encoder(**inputs).pooler_output.detach().numpy()
doc_embeddings.append(embedding)
print(doc_embeddings)
doc_embeddings = np.array(doc_embeddings)
doc_embeddings.shape[1]
doc_embeddings=np.vstack(doc_embeddings)
#create faiss index for fast retrieval
dimension=doc_embeddings.shape[1]
faiss_index=faiss.IndexFlatL2(dimension)
faiss_index.add(doc_embeddings)
#query processing and retrieval process
def retrieve_top_k(query,k=2):
  query_inputs=question_tokenizer(query,return_tensors='pt')
  query_embeddings=question_encoder(**query_inputs).pooler_output.detach().numpy()
  distances,indices=faiss_index.search(query_embeddings,k)
  retrieved_docs=[content[i] for i in indices[0]]
  return retrieved_docs
#generate the response using RAG
def generate_response(query):
  retrieved_docs=retrieve_top_k(query,k=2)
  context=" ".join(retrieved_docs)
  inputs=rag_tokenizer(f"Question:{query} context:{context}",return_tensors='pt')
  output=rag_model.generate(**inputs)
  response=rag_tokenizer.decode(output[0],skip_special_tokens=True)
  return response
#question answer bot
def chat():
  print('Hi Ask me anything or type stop to end the conversation')
  while True:
    query=input('you: ')
    if query.lower()=='stop':
      print('Goodbye')
      break
    response=generate_response(query)
    print(f'GPT: {response}')
chat()
#docs[0]
#for doc in docs:
#  print(doc.metadata)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.354 requires langchain-core<0.2,>=0.1.5, but you have langchain-core 1.2.7 which is incompatible.
langchain 0.0.354 requires langsmith<0.1.0,>=0.0.77, but you have langsmith 0.6.4 which is incompatible.
langchain-community 0.0.20 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 1.2.7 which is incompatible.
langchain-community 0.0.20 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.6.4 which is incompatible.
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cach

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This 

[array([[ 2.97106206e-01,  8.61371532e-02,  4.43722576e-01,
        -2.20220000e-01,  5.68277419e-01,  2.16011390e-01,
        -4.36428636e-01, -1.88764676e-01, -6.93817586e-02,
         5.96090704e-02, -4.13202465e-01,  5.91225088e-01,
         4.47975874e-01,  1.03446186e-01, -3.55261892e-01,
         4.46716517e-01,  5.14326155e-01,  1.67582765e-01,
        -2.48264238e-01, -6.22663796e-01, -5.60730219e-01,
         2.27026433e-01,  2.15471715e-01, -5.84067106e-01,
        -2.67453603e-02,  2.56386727e-01,  3.05207282e-01,
         1.20544694e-02, -5.17372489e-01, -5.12403175e-02,
         2.56468266e-01,  1.03034399e-01, -1.76337093e-01,
        -1.61595032e-01,  3.06515127e-01,  2.17011437e-01,
         2.68600509e-02, -5.23916960e-01, -5.82309604e-01,
        -8.48323226e-01,  2.64773548e-01,  1.28120154e-01,
        -5.61023913e-02,  5.25136948e-01, -4.48400676e-01,
        -2.89702445e-01, -1.16940618e+00, -3.53636920e-01,
        -6.95762873e-01, -1.53004542e-01,  2.10433099e-